# Import

In [1]:
from datetime import datetime
import re

import numpy as np
import pandas as pd

from db.mongo import MyMongo

# DB

In [2]:
with MyMongo() as db:
    cvs = db.get_df_from_table('cvs', 'cvs')
    distance = db.get_df_from_table('cvs', 'src_dst')

    
stop_words = ['ccup', 'mercury 전자담배', 'in cups', 'focus24', 'lgstar', 'gstop', 'sicuro', 'gs슈퍼', 'gs칼텍스', 't-cube']
stop_string = '|'.join(stop_words)

cvs = cvs.loc[~cvs['사업장명'].str.contains(stop_string, flags=re.IGNORECASE, regex=True)]

idx_tmp1 = distance['src_name'].str.contains(stop_string, flags=re.IGNORECASE, regex=True)
idx_tmp2 = distance['dst_name'].str.contains(stop_string, flags=re.IGNORECASE, regex=True)

# print(len(distance))
# print(len(distance.loc[~(idx_tmp1 | idx_tmp2), cols_distance]))

distance = distance.loc[~(idx_tmp1 | idx_tmp2)]
del idx_tmp1, idx_tmp2

<--Mongo Connected.
Mongo Connection Closed.-->


# Validate: cvs dedupe

In [3]:
print(len(cvs))
tmp = cvs.groupby('관리번호').agg({'관리번호': 'count'})
print(len(tmp))
# print(tmp.loc[tmp['관리번호']>=2])

del tmp

81458
81458


# Validate: cvs 정상영업

In [4]:
print(len(cvs.loc[cvs['상세영업상태명']=='정상영업']))

41252


# Functions

In [5]:
def add_(elem, list_):
    try:
        list_.remove(elem)
    except (KeyError, ValueError):
        pass
    finally:
        list_.append(elem)

# Columns

In [6]:
cvs.head(1)
print(cvs.columns)
cols_cvs = ['사업장명', '관리번호', 'lat', 'lng', '상세영업상태명', '도로명전체주소', '소재지전체주소', '인허가일자', '폐업일자']
cvs.loc[::, cols_cvs].head(1)
# print(distance.columns)
# cols_distance = ['src_name', 'src', 'src_status', 'src_lat', 'src_lng', 'dst_name', 'dst', 'dst_status', 'dst_lat', 'dst_lng',
#                  'line_distance', 'naver_distance']

Index(['_id', 'lat', 'lng', '개방서비스ID', '개방서비스명', '개방자치단체코드', '관리번호', '데이터갱신구분',
       '데이터갱신일자', '도로명우편번호', '도로명전체주소', '민원종류명', '번호', '사업장명', '상세영업상태명',
       '상세영업상태코드', '소재지면적', '소재지우편번호', '소재지전체주소', '소재지전화', '업태구분명', '영업상태구분코드',
       '영업상태명', '인허가일자', '인허가취소일자', '재개업일자', '좌표정보(X)', '좌표정보(Y)', '지정일자',
       '최종수정시점', '폐업일자', '휴업시작일자', '휴업종료일자'],
      dtype='object')


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자
0,세븐일레븐 종로허브점,2006300010105600037,37.57559556952654,126.98582687874092,정상영업,"서울특별시 종로구 삼일대로 461 (경운동,운현궁SK허브 101동 102호)",서울특별시 종로구 경운동 89번지 4 호 운현궁SK허브 101동 102호,20060921,NaN


# Add '개업연도' to 'cvs'

In [7]:
cvs['개업연도'] = cvs['인허가일자'].str[:4]

# cols_cvs.add('개업연도')
add_('개업연도', cols_cvs)

# print(cvs.loc[idx_active, cols_cvs].head(1))

idx_open_date_isna = cvs['개업연도'].isna()
print(len(cvs.loc[idx_open_date_isna]))
cvs.at[idx_open_date_isna, '개업연도'] = cvs['관리번호'].str[:4]

idx_open_date_isna = cvs['개업연도'].isna()
print(len(cvs.loc[idx_open_date_isna]))
cvs.loc[idx_open_date_isna, cols_cvs]

1
0


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도


# Index: outlier (<=1997)

In [8]:
idx_outlier = cvs['개업연도'].apply(lambda x: int(str(x)[:4]))<=1997
print(len(cvs.loc[idx_outlier]))
cvs.loc[idx_outlier, cols_cvs]

552


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도
68,GS25 명륜성대점,1997300007605600109,37.58788231984712,126.99658068202098,정상영업,서울특별시 종로구 명륜길 2 (명륜1가),서울특별시 종로구 명륜1가 15호,19970321,NaN,1997
418,GS25 남산조이점,2000301007105600457,37.55844042524469,126.98437692470895,정상영업,서울특별시 중구 소공로6길 24 (남산동1가),서울특별시 중구 남산동1가 18번지 1 호,19940507,NaN,1994
712,씨유 삼성화재사옥점,2000301007105600030,37.566487635370606,126.98102481848457,폐업처리,서울특별시 중구 을지로 29 (을지로1가),서울특별시 중구 을지로1가 87호,19931020,20140109.0,1993
898,약수중앙점미니스톱,2000301007105601176,37.55460636918785,127.0102443373158,폐업처리,서울특별시 중구 다산로 117 (신당동),서울특별시 중구 신당동 369번지 9호,19960525,20050205.0,1996
927,씨유 용산삼구빌딩점,2001302007405600527,37.53180334553567,126.95604377175701,정상영업,서울특별시 용산구 청파로 40 (한강로3가),서울특별시 용산구 한강로3가 16번지 49호,19911218,NaN,1991
954,씨유 후암아리랑점,2001302007405600033,37.54605974656238,126.97827796673961,정상영업,서울특별시 용산구 후암로 1 (후암동),서울특별시 용산구 후암동 194번지 8호,19850630,NaN,1985
1725,씨유(CU) 건국타운점,2000304007305600203,37.54505761314455,127.0750416220281,정상영업,서울특별시 광진구 능동로16길 35 (화양동),서울특별시 광진구 화양동 500번지 3호,19921128,NaN,1992
1746,세븐일레븐 노유점,2000304007305601097,37.536455525854066,127.06142849098518,정상영업,서울특별시 광진구 동일로 44-10 (자양동),서울특별시 광진구 자양4동 236번지 26호,19960619,NaN,1996
1773,세븐일레븐 자양점,2000304007305600326,37.53358607807819,127.07504822893948,정상영업,서울특별시 광진구 뚝섬로 585 (자양동),서울특별시 광진구 자양3동 520번지 9호,19970417,NaN,1997
2015,(주)지에스리테일 자양점,2000304007305600126,37.543574425797935,127.06969492511035,폐업처리,NaN,서울특별시 광진구 자양동 553번지 502호,19970908,20070404.0,1997


# Index: closed

In [9]:
print(cvs['상세영업상태명'].value_counts())

idx_closed = (cvs['상세영업상태명']=='폐업처리')

정상영업        41252
폐업처리        38437
지정취소         1010
직권취소          583
영업정지           96
휴업처리           62
임시소매기간만료       18
Name: 상세영업상태명, dtype: int64


# Index: active

In [10]:
idx_active = (cvs['상세영업상태명']=='정상영업')
# cvs.loc[idx_active, cols_cvs].head(1)

# Index: closed_date_isna

In [11]:
idx_closed_date_isna = cvs['폐업일자'].isna()

# Validation: 폐업일자==NaN (20)

In [12]:
print(len(cvs.loc[idx_closed&idx_closed_date_isna]))
cvs.loc[idx_closed & idx_closed_date_isna, cols_cvs]

20


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도
40694,GS25김포북변점,2005409009305600040,37.626975498656016,126.70298472775872,폐업처리,경기도 김포시 봉화로 167 (북변동),경기도 김포시 북변동 775호,20050511,NaN,2005
40788,미니스톱김포대곶점,2004409006505600017,37.65219208030824,126.58991841963676,폐업처리,NaN,경기도 김포시 대곶면 율생리 350-15호,20040219,NaN,2004
52372,GS25 포항죽도점,2006504002705600010,36.0332669683089,129.36482773351764,폐업처리,경상북도 포항시 북구 죽도로 40 (죽도동),경상북도 포항시북구 죽도동 45번지 9 호,20060310,NaN,2006
68406,(주)코리아세븐 김포2호점,2002409006505600075,37.62733031649042,126.70507223357,폐업처리,경기도 김포시 북변1로 18 (북변동),경기도 김포시 북변동 716호,20020702,NaN,2002
68408,(주)코리아세븐 김포2호점,2004409006505600035,37.62733031649042,126.70507223357,폐업처리,경기도 김포시 북변1로 18 (북변동),경기도 김포시 북변동 716호,20040326,NaN,2004
77703,훼미리마트 김포사우점,2002409006505600021,37.62080268761519,126.71948356466199,폐업처리,NaN,경기도 김포시 북변동 815호 청구.한라아파트 단지상가 1층 106호,20020306,NaN,2002
77704,LG25김포프라자,2001409006505600011,37.62253542192214,126.72068229602344,폐업처리,"경기도 김포시 사우중로73번길 38 (북변동,김포프라자)",경기도 김포시 북변동 819호 김포프라자,20010208,NaN,2001
77714,훼미리마트 김포제일점,2002409006505600046,37.62354593263838,126.71226889285451,폐업처리,"경기도 김포시 중구로 27 (북변동,1층)",경기도 김포시 북변동 858호 1층,20020501,NaN,2002
77743,LG25김포북변,2004409009305600015,37.626975498656016,126.70298472775872,폐업처리,NaN,경기도 김포시 북변동 775번지호,20040602,NaN,2004
77752,바이더웨이 오리온 마송점,2004409006505600024,37.69059747106477,126.6000243947861,폐업처리,경기도 김포시 통진읍 조강로 52,경기도 김포시 통진읍 서암리 717번지 7호,20040305,NaN,2004


# Add 'type' to 'cvs'

In [13]:

types = [
    'gs', '지에스', 'lg25', '엘지25',
     '씨유', 'cu', '훼미리마트', '훼밀리마트', '패밀리마트', 'familymart',
     '세븐일레븐', '코리아세븐', '바이더웨이', 'buytheway',
     '이마트24', 'emart24', '위드미',
     '미니스톱', '미니스탑', '대상유통',
     '씨스페이스', '씨스페이시스'
]

normalised_types = {
    'gs': 'GS25',
    '지에스': 'GS25',
    'lg25': 'GS25',
    '엘지25': 'GS25',
    
    '씨유': 'CU',
    'cu': 'CU',
    '훼미리마트': 'CU',
    '훼밀리마트': 'CU',
    '패밀리마트': 'CU',
    'familymart': 'CU',
    
    '세븐일레븐': '세븐일레븐',
    '코리아세븐': '세븐일레븐',
    '바이더웨이': '세븐일레븐',
    'buytheway': '세븐일레븐',
    
    '이마트24': '이마트24',
    'emart24': '이마트24',
    '위드미': '이마트24',
    
    '미니스톱': '미니스톱',
    '미니스탑': '미니스톱',
    '대상유통': '미니스톱',
    
    '씨스페이스': '씨스페이스',
    '씨스페이시스': '씨스페이스',
}

for idx, row in cvs.iterrows():
    name = row['사업장명'].lower()
    
    for t in types:
        if t in name:
            cvs.at[idx, '종류'] = normalised_types[t]

add_('종류', cols_cvs)
# cols_cvs.remove('종류')
# cols_cvs.append('종류')

print(len(cvs.loc[idx_closed&~idx_closed_date_isna, cols_cvs]))
print(len(cvs.loc[idx_closed&cvs['종류'].isna()]))
cvs.loc[idx_closed&~idx_closed_date_isna, cols_cvs].head(1)

38417
0


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류
219,미니스톱 혜화점,2003300007605600035,37.584080178841,127.00020831871142,폐업처리,서울특별시 종로구 창경궁로 256 (명륜2가),서울특별시 종로구 명륜2가 21번지 28호,20030422,20060605.0,2003,미니스톱


In [14]:
idx_mini = cvs['종류']=='미니스톱'
cvs.loc[idx_closed & idx_mini, cols_cvs]

,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류
219,미니스톱 혜화점,2003300007605600035,37.584080178841,127.00020831871142,폐업처리,서울특별시 종로구 창경궁로 256 (명륜2가),서울특별시 종로구 명륜2가 21번지 28호,20030422,20060605.0,2003,미니스톱
220,미니스톱 종각역점,2006300007605600007,37.57048138277121,126.9847094704115,폐업처리,"서울특별시 종로구 종로 65 (종로2가,1층 106호)",서울특별시 종로구 종로2가 8번지 4호 1층 106호,20060127,20080325.0,2006,미니스톱
237,미니스톱경희궁점,2004300007605600079,37.57356769976436,126.97215260973903,폐업처리,"서울특별시 종로구 사직로8길 34 (내수동,경희궁의아침 3단지 101호)",서울특별시 종로구 내수동 72호 경희궁의아침 3단지 101호,20040721,20151012.0,2004,미니스톱
240,미니스톱 창신역점,2006300010105600019,37.57824071676848,127.01554117342373,폐업처리,"서울특별시 종로구 지봉로 92 (숭인동, 1층)",서울특별시 종로구 숭인동 53번지 21호 1층,20060817,20061201.0,2006,미니스톱
242,미니스톱(종각중앙점),2008300012905600067,37.572311908130054,126.98256737350486,폐업처리,NaN,서울특별시 종로구 견지동 110번지 22호 1층 104호,20080716,20140808.0,2008,미니스톱
262,미니스톱 종각역점,2003300007605600086,37.57048138277121,126.9847094704115,폐업처리,서울특별시 종로구 종로 65 (종로2가),서울특별시 종로구 종로2가 8번지 4호,20030807,20060104.0,2003,미니스톱
270,미니스톱 창신역점,2003300007605600102,37.57824071676848,127.01554117342373,폐업처리,서울특별시 종로구 지봉로 92 (숭인동),서울특별시 종로구 숭인동 53번지 21호,20031001,20060803.0,2003,미니스톱
296,미니스톱 기독교회관점,2010300012905600091,37.57292586071764,127.00162102117986,폐업처리,"서울특별시 종로구 대학로 19 (연지동,기독교회관 1층)",서울특별시 종로구 연지동 136번지 46호 기독교회관 1층,20101102,20120921.0,2010,미니스톱
312,한국미니스톱(주)M경복궁역점,2012300012905600071,37.575860041203214,126.97306191821602,폐업처리,"서울특별시 종로구 사직로 지하 130 (적선동, 경복궁역사내)",NaN,20120820,20171205.0,2012,미니스톱
324,미니스톱종각중앙점,2014300016905600049,37.572311908130054,126.98256737350486,폐업처리,"서울특별시 종로구 삼봉로 95, 104호 (견지동)",서울특별시 종로구 견지동 110번지,20140829,20150819.0,2014,미니스톱


In [15]:
cvs.loc[idx_closed&cvs['종류'].isna(), cols_cvs].head()

,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류


# Add '폐업연도' to 'cvs'

In [16]:
cvs['폐업연도'] = cvs['폐업일자'].str[:4]

cvs['폐업연도'].fillna(0, inplace=True)

add_('폐업연도', cols_cvs)
# cols_cvs.add('폐업연도')

cvs.loc[~idx_closed_date_isna, cols_cvs].head(1)


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도
219,미니스톱 혜화점,2003300007605600035,37.584080178841,127.00020831871142,폐업처리,서울특별시 종로구 창경궁로 256 (명륜2가),서울특별시 종로구 명륜2가 21번지 28호,20030422,20060605.0,2003,미니스톱,2006


# 기준연도 = 2008

In [17]:
criteria_year = 2008

# Index: close_after_criteria

In [47]:
print(cvs['상세영업상태명'].value_counts())
idx_close_after_criteria = idx_closed & (cvs['폐업연도'].apply(lambda x: int(str(x)[:4]))>=criteria_year)
#                             & (cvs['개업연도'].apply(lambda x: int(str(x)[:4]))>=1989)
# idx_closed = (cvs['상세영업상태명']=='폐업처리') & idx_close_after_1998

print(len(cvs.loc[idx_close_after_criteria]))
cvs.loc[idx_close_after_criteria, cols_cvs].head(5)

# cvs.loc[~idx_close_after_1998, cols_cvs]

# cvs.loc[idx_close_after_criteria, cols_cvs].to_csv('close_after_2008.tsv', sep='\t', index=False)

정상영업        41252
폐업처리        38437
지정취소         1010
직권취소          583
영업정지           96
휴업처리           62
임시소매기간만료       18
Name: 상세영업상태명, dtype: int64
31639


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도,period
220,미니스톱 종각역점,2006300007605600007,37.57048138277121,126.9847094704115,폐업처리,"서울특별시 종로구 종로 65 (종로2가,1층 106호)",서울특별시 종로구 종로2가 8번지 4호 1층 106호,20060127,20080325.0,2006,미니스톱,2008,788.0
221,GS25 종로용비점,2006300007605600008,37.57343100708293,126.97327108221329,폐업처리,"서울특별시 종로구 새문안로3길 36 (내수동,용비어천가빌딩 1층 101호)",서울특별시 종로구 내수동 75호 용비어천가빌딩 1층 101호,20060203,20100225.0,2006,GS25,2010,1483.0
223,GS25종로리버,2008300012905600064,37.568838018022426,126.98510824788185,폐업처리,"서울특별시 종로구 청계천로 67, 104호 (관철동,종로1번가 1층)",서울특별시 종로구 관철동 155번지 종로1번가 1층 104호,20080709,20140502.0,2008,GS25,2014,2123.0
237,미니스톱경희궁점,2004300007605600079,37.57356769976436,126.97215260973903,폐업처리,"서울특별시 종로구 사직로8길 34 (내수동,경희궁의아침 3단지 101호)",서울특별시 종로구 내수동 72호 경희궁의아침 3단지 101호,20040721,20151012.0,2004,미니스톱,2015,4100.0
238,씨유 종로효제점,1999300007605600163,37.57148515262966,127.00157594371186,폐업처리,NaN,서울특별시 종로구 효제동 291번지 (주)훼미리마트 종로점,19990723,20171101.0,1999,CU,2017,6676.0


# Index: open_after_criteria

In [19]:
print(len(cvs['상세영업상태명']))

idx_open_after_criteria = (cvs['개업연도'].apply(lambda x: int(str(x)[:4]))>=criteria_year)
# idx_closed = (cvs['상세영업상태명']=='폐업처리') & idx_close_after_1998

print(len(cvs.loc[idx_open_after_criteria]))
cvs.loc[idx_open_after_criteria, cols_cvs].head(1)

# cvs.loc[~idx_close_after_1998, cols_cvs]

81458
63048


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도
4,세븐일레븐 종로배화점,2008300012905600082,37.57716379872892,126.96943884968033,정상영업,서울특별시 종로구 필운대로 12 (필운동),서울특별시 종로구 필운동 163번지,20081007,NaN,2008,세븐일레븐,0


# -----

# 업체별 폐업

In [20]:
print(sum(cvs.loc[idx_close_after_criteria].groupby('종류').agg({'관리번호':'count'})['관리번호']))
close_corp = cvs.loc[idx_close_after_criteria].groupby('종류').agg({'관리번호':'count'})
close_corp

31622


,관리번호
종류,
CU,10095
GS25,11357
미니스톱,3271
세븐일레븐,5835
씨스페이스,148
이마트24,916


# 업체별 개업

In [21]:
print(sum(cvs.loc[idx_open_after_criteria].groupby('종류').agg({'관리번호':'count'})['관리번호']))
open_corp = cvs.loc[idx_open_after_criteria].groupby('종류').agg({'관리번호':'count'})
open_corp

63048


,관리번호
종류,
CU,19495
GS25,21293
미니스톱,5028
세븐일레븐,12830
씨스페이스,299
이마트24,4103


# 업체별 폐업 / 개업

In [22]:
close_corp / open_corp * 100

,관리번호
종류,
CU,51.782508
GS25,53.336777
미니스톱,65.055688
세븐일레븐,45.479345
씨스페이스,49.498328
이마트24,22.325128


# -----

# Validate '개업연도'

In [23]:
# 인허가일자 앞 4자리와 관리번호 앞 4자리 비교, 다른지 확인
# 왜냐하면 인허가일자가 없는 가게가 있기 때문, 관리번호는 모든 가게가 갖고 있으므로
# 나는 지금 관리번호 앞 4자리로, 개업연도를 도출하려고 한다.

# 담당공무원과 전화통화 결과 인허가일자를 개업일자로 보면될듯 하다.

cols_tmp = ['사업장명', '상세영업상태명', '관리번호', '폐업일자', '인허가일자', '인허가취소일자', '재개업일자', '지정일자', '최종수정시점']

print(len(cvs.loc[cvs['인허가일자'].str[:4]!=cvs['관리번호'].str[:4], cols_tmp]))
cvs.loc[cvs['인허가일자'].str[:4]!=cvs['관리번호'].str[:4], cols_tmp]

834


,사업장명,상세영업상태명,관리번호,폐업일자,인허가일자,인허가취소일자,재개업일자,지정일자,최종수정시점
352,미니스톱 종로6가점,폐업처리,2004300007605600003,20110207.0,20100222,NaN,NaN,20100222,20110208141310
417,세븐일레븐 종로창신제일점,직권취소,1995300007605600127,NaN,20000215,20170912.0,NaN,20000215,20170912212202
418,GS25 남산조이점,정상영업,2000301007105600457,NaN,19940507,NaN,NaN,19940507,20160718114945
431,세븐일레븐 중구오장점,정상영업,2000301007105600565,NaN,19981112,NaN,NaN,19981112,20131024153638
707,지에스칼텍스(주) 역전주유소,폐업처리,2000301007105600320,20060117.0,19981218,NaN,NaN,NaN,20080221000000
712,씨유 삼성화재사옥점,폐업처리,2000301007105600030,20140109.0,19931020,NaN,NaN,19931020,20140109211340
715,미니스톱남산지점,폐업처리,2000301007105600484,20090624.0,19991108,NaN,NaN,19991108,20090624160503
898,약수중앙점미니스톱,폐업처리,2000301007105601176,20050205.0,19960525,NaN,NaN,NaN,20080221000000
899,미니스톱신당점,폐업처리,2000301007105601185,20021005.0,19991210,NaN,NaN,NaN,20080221000000
904,세븐일레븐(명동 4호점),폐업처리,2000301007105600513,20090630.0,19991021,NaN,NaN,19991021,20090630152739


# Add 'period' to 'cvs'

In [24]:
# def parse_date_string(x):
#     try:
#         return datetime.strptime(str(x)[:8], '%Y%m%d')
#     except ValueError:
#         return np.nan

def get_period(row):
    if (row['인허가일자'] == np.nan) or (row['폐업일자'] == np.nan):
        return np.nan
    
    try:
        open_date = datetime.strptime(str(row['인허가일자'])[:8], '%Y%m%d')
        close_date = datetime.strptime(str(row['폐업일자'])[:8], '%Y%m%d')
        return (close_date - open_date).days
    except ValueError:
        print(row['인허가일자'], row['폐업일자'])
    

cvs.at[idx_closed &~idx_closed_date_isna, 'period'] = cvs.loc[idx_closed &~idx_closed_date_isna].apply(get_period, axis=1)

# cols_cvs.add('period')
add_('period', cols_cvs)

00000000 20030205.0
1990521  20020424.0
199905   20051115.0
199905   20050602.0
199905   20050909.0
199905   20050112.0
199905   20050111.0
199905   20050111.0
199908   20010727.0
  000616 20050901.0
  010427 20041102.0
20000000 20020528.0
19000000 20060224.0
0        20050729.0


# -----

# 연도별 폐업 Group 'cvs' by 'close_year'

In [25]:
close_year = cvs.loc[idx_close_after_criteria&~idx_closed_date_isna, cols_cvs].groupby('폐업연도').agg({'관리번호':'count'})

close_year = close_year.rename(columns={'관리번호':'편의점 수'})

close_year = close_year.sort_values('폐업연도', ascending=False)
close_year
# Export
# tmp.sort_values('편의점 수', ascending=False).reset_index().to_csv('closed_after_2008_groupby_year.tsv', sep='\t', index=False)

,편의점 수
폐업연도,
2018,4270
2017,3987
2016,3550
2015,3215
2014,2947
2013,3219
2012,2768
2011,2177
2010,1924


In [26]:
idx_2013 = cvs['폐업연도']=='2013'
cvs.loc[idx_closed&~idx_closed_date_isna&idx_2013, cols_cvs].head(1)

,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도,period
248,씨유 원남사거리점,2009300012905600084,37.575625234497465,126.99795780184026,폐업처리,"서울특별시 종로구 율곡로 174 (원남동,1층)",서울특별시 종로구 원남동 28번지 7호 1층,20091103,20131031.0,2009,CU,2013,1458.0


# 연도별 개업 Group 'cvs' by 'open_year'

In [67]:
open_year = cvs.loc[idx_open_after_criteria].groupby('개업연도').agg({'관리번호':'count'})

open_year = open_year.rename(columns={'관리번호':'편의점 수'})

open_year = open_year.sort_values('개업연도', ascending=False)
open_year

# Export
# open_year.reset_index().to_csv('open_year.tsv', sep='\t', index=False)
# tmp.sort_values('편의점 수', ascending=False).reset_index().to_csv('closed_cvs_groupby_year.tsv', sep='\t', index=False)

# 연도별 폐업 / 개업

In [28]:
close_year / open_year * 100

,편의점 수
폐업연도,
2018,62.600792
2017,45.337730
2016,44.755421
2015,47.785375
2014,63.787879
2013,74.808273
2012,46.202637
2011,35.209445
2010,41.645022


# -----

# 연도&업체별 폐업 Group 'cvs' by 'closed_year' & 'corp'

In [29]:
close_per_year_corp = cvs.loc[idx_close_after_criteria&~idx_closed_date_isna, cols_cvs].pivot_table(columns='종류', values='관리번호',
                                                                                                    aggfunc='count', index='폐업연도')
close_per_year_corp = close_per_year_corp.reset_index().fillna(0); close_per_year_corp

# Export
# close_per_year_corp.to_csv('close_per_year_corp.tsv', sep='\t', index=False)

# .groupby('종류').agg({'관리번호':'count'})

종류,폐업연도,CU,GS25,미니스톱,세븐일레븐,씨스페이스,이마트24
0,2008,741,642,257,164,8,15
1,2009,740,578,178,212,15,15
2,2010,757,665,242,234,9,17
3,2011,732,902,196,320,12,15
4,2012,968,1102,214,455,10,19
5,2013,1135,1108,305,656,8,7
6,2014,857,1075,353,624,17,21
7,2015,924,1185,349,666,8,83
8,2016,1015,1159,370,816,18,172
9,2017,1097,1335,396,890,20,249


## 2013년 월별 CU 폐업

In [30]:
idx_cu = cvs['종류']=='CU'
idx_2013 = cvs['폐업연도']=='2013'
tmp = cvs.loc[idx_closed& ~idx_closed_date_isna& idx_cu& idx_2013, cols_cvs]
tmp['폐업달'] = tmp['폐업일자'].str[4:6]; tmp.head()
cu_2013 = tmp.groupby('폐업달').agg({'관리번호':'count'})
cu_2013 = cu_2013.rename(columns={'관리번호':'폐업 편의점 수'})
cu_2013 = cu_2013.reset_index()
cu_2013

# cu_2013.to_csv('cu_2013.tsv', sep='\t', index=False)

,폐업달,폐업 편의점 수
0,01,71
1,02,69
2,03,85
3,04,70
4,05,93
5,06,110
6,07,116
7,08,129
8,09,105
9,10,95


# 연도&업체별 개업 Group 'cvs' by 'open_year' & 'corp'

In [31]:
open_per_year_corp = cvs.loc[idx_open_after_criteria, cols_cvs].pivot_table(columns='종류', values='관리번호', aggfunc='count', index='개업연도')

open_per_year_corp = open_per_year_corp.reset_index().fillna(0); open_per_year_corp

# open_per_year_corp.loc[open_per_year_corp['개업연도'].apply(int) >= 2000]
open_per_year_corp
# print(len(cvs.loc[idx_active, cols_cvs]))
# cvs.loc[idx_active, cols_cvs].head(5)

종류,개업연도,CU,GS25,미니스톱,세븐일레븐,씨스페이스,이마트24
0,2008,1224,1206,363,558,19,58
1,2009,1283,1199,337,731,32,46
2,2010,1425,1594,415,1106,19,61
3,2011,1947,2115,471,1553,14,83
4,2012,2115,1934,426,1428,20,68
5,2013,1283,1716,385,830,6,83
6,2014,1334,1668,450,887,9,272
7,2015,2053,2163,575,1434,19,484
8,2016,2448,2511,575,1569,39,790
9,2017,2589,2923,528,1622,61,1071


## 1995년 이전에 개업?

In [32]:
cvs['개업연도'].fillna('0', inplace=True)
cvs.loc[cvs['개업연도'].apply(int)<=1995, cols_cvs]

,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도,period
418,GS25 남산조이점,2000301007105600457,37.55844042524469,126.98437692470895,정상영업,서울특별시 중구 소공로6길 24 (남산동1가),서울특별시 중구 남산동1가 18번지 1 호,19940507,NaN,1994,GS25,0,NaN
712,씨유 삼성화재사옥점,2000301007105600030,37.566487635370606,126.98102481848457,폐업처리,서울특별시 중구 을지로 29 (을지로1가),서울특별시 중구 을지로1가 87호,19931020,20140109.0,1993,CU,2014,7386.0
927,씨유 용산삼구빌딩점,2001302007405600527,37.53180334553567,126.95604377175701,정상영업,서울특별시 용산구 청파로 40 (한강로3가),서울특별시 용산구 한강로3가 16번지 49호,19911218,NaN,1991,CU,0,NaN
954,씨유 후암아리랑점,2001302007405600033,37.54605974656238,126.97827796673961,정상영업,서울특별시 용산구 후암로 1 (후암동),서울특별시 용산구 후암동 194번지 8호,19850630,NaN,1985,CU,0,NaN
1725,씨유(CU) 건국타운점,2000304007305600203,37.54505761314455,127.0750416220281,정상영업,서울특별시 광진구 능동로16길 35 (화양동),서울특별시 광진구 화양동 500번지 3호,19921128,NaN,1992,CU,0,NaN
2026,세븐일레븐구의4호점,2000304007305600997,NaN,NaN,폐업처리,NaN,서울특별시 광진구 구의동 221번지 40호,19950301,20040930.0,1995,세븐일레븐,2004,3501.0
2032,미니스톱,2000304007305600162,37.49557794378786,127.03185020982899,폐업처리,NaN,서울특별시 광진구 중곡동 18번지 73호,19940916,20011227.0,1994,미니스톱,2001,2659.0
2396,서울시립대학교 씨유 학생회관점,1989305008205608028,37.582576494365554,127.05999005076032,정상영업,서울특별시 동대문구 서울시립대로 163 (전농동),서울특별시 동대문구 전농동 90호,19890419,NaN,1989,CU,0,NaN
2399,지에스25 고대으뜸,1991305008205605031,37.588205165955294,127.03491795373104,정상영업,서울특별시 동대문구 안암로22길 2 (제기동),서울특별시 동대문구 제기동 137번지 66호,19910429,NaN,1991,GS25,0,NaN
3639,씨유 수유중앙점,1989308007505600690,37.63867752469118,127.02473554394837,정상영업,서울특별시 강북구 한천로139길 46 (수유동),서울특별시 강북구 수유동 192번지 4 호,19890809,NaN,1989,CU,0,NaN


## 전체 GS25 개수?

In [33]:
cvs.loc[idx_active].groupby('종류').agg({'관리번호':'count'})

,관리번호
종류,
CU,12809
GS25,12807
미니스톱,2661
세븐일레븐,9375
씨스페이스,217
이마트24,3383


# 연도&업체별 폐업/개업 비율

In [34]:
# idx_close_2000 = close_per_year_corp['폐업연도'].apply(int)>=2000
# idx_open_2000 = open_per_year_corp['개업연도'].apply(int)>=2000

cols_ratio = ['CU', 'GS25', '미니스톱', '세븐일레븐', '씨스페이스', '이마트24']

tmp = close_per_year_corp.rename(columns={'폐업연도': '연도'}).reset_index().drop(columns=['index']).applymap(float)
tmp2 = open_per_year_corp.rename(columns={'개업연도': '연도'}).reset_index().drop(columns=['index']).applymap(float)
ratio_2000 = tmp / tmp2 * 100
ratio_2000['연도'] = range(criteria_year, 2019)

cols_ratio.insert(0, '연도')

ratio_2000.loc[::, cols_ratio]



종류,연도,CU,GS25,미니스톱,세븐일레븐,씨스페이스,이마트24
0,2008,60.539216,53.233831,70.798898,29.390681,42.105263,25.862069
1,2009,57.677319,48.206839,52.818991,29.001368,46.875000,32.608696
2,2010,53.122807,41.718946,58.313253,21.157324,47.368421,27.868852
3,2011,37.596302,42.647754,41.613588,20.605280,85.714286,18.072289
4,2012,45.768322,56.980352,50.234742,31.862745,50.000000,27.941176
5,2013,88.464536,64.568765,79.220779,79.036145,133.333333,8.433735
6,2014,64.242879,64.448441,78.444444,70.349493,188.888889,7.720588
7,2015,45.007306,54.785021,60.695652,46.443515,42.105263,17.148760
8,2016,41.462418,46.156910,64.347826,52.007648,46.153846,21.772152
9,2017,42.371572,45.672255,75.000000,54.870530,32.786885,23.249300


In [35]:
close_per_year_corp

종류,폐업연도,CU,GS25,미니스톱,세븐일레븐,씨스페이스,이마트24
0,2008,741,642,257,164,8,15
1,2009,740,578,178,212,15,15
2,2010,757,665,242,234,9,17
3,2011,732,902,196,320,12,15
4,2012,968,1102,214,455,10,19
5,2013,1135,1108,305,656,8,7
6,2014,857,1075,353,624,17,21
7,2015,924,1185,349,666,8,83
8,2016,1015,1159,370,816,18,172
9,2017,1097,1335,396,890,20,249


In [36]:
open_per_year_corp

종류,개업연도,CU,GS25,미니스톱,세븐일레븐,씨스페이스,이마트24
0,2008,1224,1206,363,558,19,58
1,2009,1283,1199,337,731,32,46
2,2010,1425,1594,415,1106,19,61
3,2011,1947,2115,471,1553,14,83
4,2012,2115,1934,426,1428,20,68
5,2013,1283,1716,385,830,6,83
6,2014,1334,1668,450,887,9,272
7,2015,2053,2163,575,1434,19,484
8,2016,2448,2511,575,1569,39,790
9,2017,2589,2923,528,1622,61,1071


## 미니스톱 개수?

In [37]:
idx_mini = cvs['종류']=='미니스톱'
print(len(cvs.loc[idx_closed &idx_mini, cols_cvs]))
cvs.loc[idx_close_after_criteria &idx_mini, cols_cvs]

# 7376 - 4544 = 2832

4604


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도,period
220,미니스톱 종각역점,2006300007605600007,37.57048138277121,126.9847094704115,폐업처리,"서울특별시 종로구 종로 65 (종로2가,1층 106호)",서울특별시 종로구 종로2가 8번지 4호 1층 106호,20060127,20080325.0,2006,미니스톱,2008,788.0
237,미니스톱경희궁점,2004300007605600079,37.57356769976436,126.97215260973903,폐업처리,"서울특별시 종로구 사직로8길 34 (내수동,경희궁의아침 3단지 101호)",서울특별시 종로구 내수동 72호 경희궁의아침 3단지 101호,20040721,20151012.0,2004,미니스톱,2015,4100.0
242,미니스톱(종각중앙점),2008300012905600067,37.572311908130054,126.98256737350486,폐업처리,NaN,서울특별시 종로구 견지동 110번지 22호 1층 104호,20080716,20140808.0,2008,미니스톱,2014,2214.0
296,미니스톱 기독교회관점,2010300012905600091,37.57292586071764,127.00162102117986,폐업처리,"서울특별시 종로구 대학로 19 (연지동,기독교회관 1층)",서울특별시 종로구 연지동 136번지 46호 기독교회관 1층,20101102,20120921.0,2010,미니스톱,2012,689.0
312,한국미니스톱(주)M경복궁역점,2012300012905600071,37.575860041203214,126.97306191821602,폐업처리,"서울특별시 종로구 사직로 지하 130 (적선동, 경복궁역사내)",NaN,20120820,20171205.0,2012,미니스톱,2017,1933.0
324,미니스톱종각중앙점,2014300016905600049,37.572311908130054,126.98256737350486,폐업처리,"서울특별시 종로구 삼봉로 95, 104호 (견지동)",서울특별시 종로구 견지동 110번지,20140829,20150819.0,2014,미니스톱,2015,355.0
338,한국미니스톱(주) M안국역점,2012300012905600072,37.57660162388654,126.98512960517377,폐업처리,"서울특별시 종로구 율곡로 지하 53 (안국동, 안국역사내)",NaN,20120820,20171205.0,2012,미니스톱,2017,1933.0
352,미니스톱 종로6가점,2004300007605600003,37.57143364267875,127.00654281806953,폐업처리,서울특별시 종로구 종로41길 3 (종로6가),서울특별시 종로구 종로6가 247번지 1호,20100222,20110207.0,2010,미니스톱,2011,350.0
357,미니스톱 숭인지봉로점,2015300016905600029,37.57757397690119,127.01557273276373,폐업처리,서울특별시 종로구 지봉로 84 (숭인동),서울특별시 종로구 숭인동 55번지 5호,20150413,20170630.0,2015,미니스톱,2017,809.0
374,미니스톱 숭인창신점,2015300016905600039,NaN,NaN,폐업처리,서울특별시 종로구 지봉로 100-11 (숭인동),서울특별시 종로구 숭인동 9번지 2호,20150518,20170616.0,2015,미니스톱,2017,760.0


# -----

# 얼마나 버텼을까?

In [48]:
cvs.loc[idx_open_after_criteria &idx_close_after_criteria &~idx_closed_date_isna, cols_cvs].sort_values('개업연도', ascending=True).head(50)

,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,인허가일자,폐업일자,개업연도,종류,폐업연도,period
223,GS25종로리버,2008300012905600064,37.568838018022426,126.98510824788185,폐업처리,"서울특별시 종로구 청계천로 67, 104호 (관철동,종로1번가 1층)",서울특별시 종로구 관철동 155번지 종로1번가 1층 104호,20080709,20140502.0,2008,GS25,2014,2123.0
61724,GS25창원기산,2008527018205600117,35.22362545974828,128.67892906828345,폐업처리,"경상남도 창원시 성산구 중앙대로 95 (중앙동,기산파라다이스 110,111호)","경상남도 창원시 중앙동 96번지 1호 기산파라다이스 110,111호",20080910,20130823.0,2008,GS25,2013,1808.0
61723,GS25 봉곡제일점,2008527018205600152,35.24362114899295,128.65120097160954,폐업처리,경상남도 창원시 의창구 지귀로12번길 21 (봉곡동),경상남도 창원시 봉곡동 33번지 7호,20081208,20081216.0,2008,GS25,2008,8.0
8932,GS25사당드림점,2008319009905600106,37.47852063420198,126.97919294480425,폐업처리,"서울특별시 동작구 동작대로7길 56 (사당동,1층)",서울특별시 동작구 사당동 1038번지 51호 1층,20081205,20091109.0,2008,GS25,2009,339.0
8939,GS25 사당까치,2008319009905600081,37.48086539300482,126.97206946584694,폐업처리,서울특별시 동작구 사당로16길 65 (사당동),서울특별시 동작구 사당동 309번지 21호,20080919,20100805.0,2008,GS25,2010,685.0
61722,씨유 소답스마일점,2008527018205600144,35.26638495726908,128.6267979295447,폐업처리,"경상남도 창원시 의창구 의안로56번길 26, 101호 (소답동, 북창상가)",경상남도 창원시 소답동 130번지 1호 북창상가 101호,20081117,20130802.0,2008,CU,2013,1719.0
61721,(주)지에스리테일 창원남양점,2008527018205600140,35.21262033228109,128.70160477427618,폐업처리,"경상남도 창원시 성산구 가음로15번길 38, 지하2층 (남양동, 제운동시설동)",경상남도 창원시 성산구 22번지 제운동시설동 지하2층,20081031,20180823.0,2008,GS25,2018,3583.0
8964,미니스톱 노량진미도점,2008319009905600043,37.51331429668576,126.94618692744572,폐업처리,서울특별시 동작구 노량진로 184 (노량진동),서울특별시 동작구 노량진동 148번지 5호,20080604,20110419.0,2008,미니스톱,2011,1049.0
8965,GS25 동작대방점,2008319009905600046,37.499538087034125,126.9293782329336,폐업처리,"서울특별시 동작구 상도로 80 (대방동,1층)",서울특별시 동작구 대방동 336번지 13호 1층,20080610,20090604.0,2008,GS25,2009,359.0
8968,세븐일레븐(상도신동아점),2008319009905600004,37.50568847923075,126.94681670365591,폐업처리,서울특별시 동작구 만양로 9-1 (상도동),서울특별시 동작구 상도동 20번지 30호,20080118,20150403.0,2008,세븐일레븐,2015,2632.0


In [49]:
cvs.loc[idx_open_after_criteria &idx_close_after_criteria &~idx_closed_date_isna]['period'].mean()

1025.7155866758408

In [50]:
cvs.loc[idx_open_after_criteria &idx_close_after_criteria &~idx_closed_date_isna, cols_cvs].groupby('종류').agg({'period':'mean'})

,period
종류,
CU,1034.230759
GS25,968.426667
미니스톱,1086.710262
세븐일레븐,1173.422012
씨스페이스,956.752381
이마트24,629.565632


In [41]:
365*5

1825

In [60]:
1026 / 365

2.8109589041095893

In [61]:
12*0.81

9.72

In [51]:
print(len(cvs.loc[idx_open_after_criteria &idx_close_after_criteria &~idx_closed_date_isna]))

24707


# x일 못 버틴 편의점, x=365

In [62]:
limit = 730

In [63]:
idx_close_within_limit = cvs['period']<=limit

# idx_open_5_years_ago = cvs['개업연도'].apply(int)>=2012

cvs_close_within_limit = cvs.loc[idx_open_after_criteria &idx_close_after_criteria & ~idx_closed_date_isna
                            & idx_close_within_limit, cols_cvs].groupby('종류').agg({'관리번호':'count'})
cvs_close_within_limit

,관리번호
종류,
CU,3127
GS25,4288
미니스톱,956
세븐일레븐,1530
씨스페이스,52
이마트24,612


In [53]:
cvs_open_since_limit = cvs.loc[idx_open_after_criteria].groupby('종류').agg({'관리번호':'count'})
cvs_open_since_limit

,관리번호
종류,
CU,19495
GS25,21293
미니스톱,5028
세븐일레븐,12830
씨스페이스,299
이마트24,4103


In [64]:
close_within_year = cvs_close_within_limit / cvs_open_since_limit * 100
close_within_year = close_within_year.reset_index()
close_within_year = close_within_year.rename(columns={'관리번호':'1년 내 폐업 점포 비율'})
close_within_year

# close_within_year.to_csv('close_within_year.tsv', sep='\t', index=False)

,종류,1년 내 폐업 점포 비율
0,CU,16.040010
1,GS25,20.138074
2,미니스톱,19.013524
3,세븐일레븐,11.925175
4,씨스페이스,17.391304
5,이마트24,14.915915
